### import libraries

In [1]:
using CSV
using DataFrames

import Base.Threads.@spawn

include("util.jl")

remove (generic function with 2 methods)

Maps names to a set of corresponding person_ids

In [2]:
names = Dict{String, Set}();

Maps person_ids to a dictionary of: name, birth, movies (a set of movie_ids)

In [3]:
people = Dict{Int64, Dict{String, Any}}();

Maps movie_ids to a dictionary of: title, year, stars (a set of person_ids)

In [4]:
movies = Dict{Int64, Dict{String, Any}}();

In [5]:
"""Load data from CSV files into memory. """
function load_data(directory::String)
    # Load people
    people_data = CSV.read(directory*"/people.csv");
    for row ∈ 1:nrow(people_data)
        people[people_data[row, 1]] = Dict(
            "name" => people_data[row, 2], 
            "birth" => people_data[row, 3], 
            "movies" => Set()
        );
        if !(haskey(names, lowercase(people_data[row, 2])))
            names[lowercase(people_data[row, 2])] = Set(people_data[row, 1]);
        else
            push!(names[lowercase(people_data[row, 2])], people_data[row, 1]);
        end
    end

    # Load movies
    movies_data = CSV.read(directory*"/movies.csv");
    for row ∈ 1:nrow(movies_data)
        movies[movies_data[row, 1]] = Dict(
            "title" => movies_data[row, 2], 
            "year" => movies_data[row, 3], 
            "stars" => Set()
        );
    end

    # Load stars
    stars_data = CSV.read(directory*"/stars.csv");
    for row ∈ 1:nrow(stars_data)
        try 
            push!(people[stars_data[row, 1]]["movies"], stars_data[row, 2]);
            push!(movies[stars_data[row, 2]]["stars"], stars_data[row, 1]);
        catch KeyError
            # pass
        end
    end
end

load_data

In [6]:
"""
Returns (movie_id, person_id) pairs for people
    who starred with a given person. 
"""
function neighbors_for_person(person_id :: Int64)
    movie_ids = people[person_id]["movies"];
    neighbors = Set();
    for movie_id ∈ movie_ids, person_id ∈ movies[movie_id]["stars"]
        push!(neighbors, (movie_id, person_id));
    end
    return neighbors;
end

neighbors_for_person

In [7]:
"""
Returns the IMDB id for a person's name,
resolving ambiguities as needed. 
"""
function person_id_for_name(name::String)
    person_ids = collect(get(names, lowercase(name), Set()));
    if length(person_ids) == 0
        return nothing;
    elseif length(person_ids) > 1
        print("Which '$name'?");
        for person_id ∈ person_ids
            person = people[person_id];
            name, birth = person["name"], person["birth"];
            println("ID: $person_id, Name: $name, birth: $birth");
        end
        try 
            print("Intended Person ID: ");
            person_id = readline();
            if person_id ∈ person_ids
                return person_id;
            end
        catch ArgumentError
            # pass
        end
        return nothing;
    else
        return first(person_ids);
    end
end

person_id_for_name

In [15]:
"""
Returns the shortest list of (movie_id, person_id) pairs
    that connect the source to the target.

    If no possible path, returns nothing. 
"""
function shortest_path(source :: Int64, target :: Int64)
    
    # Keep track of number of states explored
    num_explored = 0

    # Initialize frontier to just the starting position
    start, goal = Node(source, nothing, nothing), Node(target, nothing, nothing)

    frwd_frontier, bcwd_frontier =  [start], [goal]
    frontier_type = typeof(frwd_frontier)

    # Initialize an empty explored set
    frwd_explored, bcwd_explored = Set{Union{Nothing, Integer}}(), Set{Union{Nothing, Integer}}()
    frwd_path, bcwd_path = Tuple{Union{Nothing, Integer, Node}, Union{Nothing, Integer, Node}}[], Tuple{Union{Nothing, Integer, Node}, Union{Nothing, Integer, Node}}[]

    # Keep looping until solution found
    while true
        # If nothing left in frontier, then no path
        if isempty(frwd_frontier) || isempty(bcwd_frontier)
            return frwd_path, true;
        end

        # Choose a node from the frontier
        frwd_node, bcwd_node = popfirst!(frwd_frontier), popfirst!(bcwd_frontier);
        num_explored += 1;

        # Add neighbors to forward frontier
        for (action, state) in neighbors_for_person(frwd_node.state)
            if !(state in frwd_explored)
                child = Node(state, frwd_node, action);
                # If child is the target, then we have a solution
                if child.state == target
                    while child.parent != nothing
                        pushfirst!(frwd_path, (child.action, child.state))
                        child = child.parent
                    end
                    return frwd_path, true
                end
                push!(frwd_frontier, child);
                push!(frwd_explored, child.state);
            end
        end
        
        # Add neighbors to backward frontier
        for (action, state) in neighbors_for_person(bcwd_node.state)
            if !(state in bcwd_explored)
                child = Node(state, bcwd_node, action);
                # If child is the target, then we have a solution
                if child.state == source
                    while child.parent != nothing
                        pushfirst!(bcwd_path, (child.action, child.state))
                        child = child.parent
                    end
                    return bcwd_path, false
                end
                push!(bcwd_frontier, child);
                push!(bcwd_explored, child.state);
            end
        end
    end
end

shortest_path

how mny threads does julia have?

In [9]:
Threads.nthreads()

1

## start

get data directory

In [10]:
directory = "large";

##### Load data from files into memory

it might take some time to load the data as it's large 😅

In [11]:
println("Loading data...");
load_data(directory);
println("Data loaded.");

Loading data...
Data loaded.


get person

In [18]:
print("Enter Person 1 Name: ");
source =  person_id_for_name(readline());
if (source == nothing)
    println("Person not found.");
    exit(1)
end
print("Enter Person 2 Name: ");
target = person_id_for_name(readline())
if (target == nothing)
    println("Person not found.");
    exit(1)
end

Enter Person 1 Name: 

stdin>  Juliane Banse


Enter Person 2 Name: 

stdin>  Julian Acosta


find the shortest path between them

In [ ]:
@time path, used_frwd_path = shortest_path(source, target);

find connection between person1(source) and person2(target)

In [17]:
if isempty(path)
    println("Not connected.");
else
    degrees = length(path);
    println("$degrees degrees of seperation.");
    path = vcat([(nothing, if used_frwd_path source else target end)], path);
    for i ∈ 1:degrees
        person1 = people[path[i][2]]["name"];
        person2 = people[path[i + 1][2]]["name"];
        movie = movies[path[i + 1][1]]["title"];
        println("$(i): $person1 and $person2 starred in $movie");
    end
end

Not connected.
